In [1]:
%pip install llama-index-llms-huggingface

In [2]:
!pip install llama-index

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM

In [4]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [5]:
# load documents
documents = SimpleDirectoryReader("./data/").load_data()

In [6]:
print(documents)

[Document(id_='1af5da09-8c33-4464-9e07-fbd3ed841840', embedding=None, metadata={'page_label': '3093', 'file_name': 'Machine-translation.pdf', 'file_path': '/content/data/Machine-translation.pdf', 'file_type': 'application/pdf', 'file_size': 470409, 'creation_date': '2025-10-21', 'last_modified_date': '2025-10-21'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Proceedings of COLING 2016, the 26th International Conference on Computational Linguistics: Technical Papers,\npages 3093–3102, Osaka, Japan, December 11-17 2016.\nNeural Machine Translation with Supervised Attention\nLemao Liu, Masao Utiyama, Andrew Finch and Eiichiro Sumita\nNati

In [7]:
# set up prompts - specific to stable LM
from llama_index.core import PromptTemplate

system_prompt = """<|SYSTEM|># You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided.
"""

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [8]:
# loading the model in the memoery itself
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    stopping_ids=[50278,50279,20277,1,0],
    tokenizer_kwargs={"max_length": 4096},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    # Uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16} if torch.cuda.is_available() else {}, #kwargs-keyword arguement
)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
!pip install llama-index-embeddings-instructor
!pip install llama-index-embeddings-huggingface

In [10]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding(model_name='sentence-transformers/all-mpnet-base-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
from llama_index.core import VectorStoreIndex, Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 1024

In [14]:
index = VectorStoreIndex.from_documents(
    documents, embed_model=embed_model
)

In [15]:
query_engine = index.as_query_engine(llm=llm)

In [19]:
query_engine.query("what is attention")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 160.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 42.12 MiB is free. Process 168881 has 14.70 GiB memory in use. Of the allocated memory 14.46 GiB is allocated by PyTorch, and 110.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)